In [1]:
import pennylane as qml
from autograd import grad

In [2]:
################ Code from the 'Optimization of molecular geometries' tutorial##################
# imports and relevant defines:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

# Simulation starting parameters:
symbols = ["O", "H", "H", "O", "H", "H"]

# np.array([ 0.        ,  0.7581    , -0.5086    , -1.49139892,  0.29819887,
#        -1.2393812 , -0.68850404,  2.16155953,  0.21671681], requires_grad=True)

# this is selected from the tutorial 'Building molecular Hamiltonians', which has the same nulcear-coordinats for water
x = np.array([ 0.        ,  0.        ,  0.        ,  1.5633147 ,  0.98274071,
        -0.60341695, -0.7834994 ,  1.43656484,  1.04702164,  1.88208385,
         1.81779458, -0.08610321,  2.16704842,  2.60039367,  1.66894039,
         2.22750225,  3.42941152, -1.11431581], requires_grad=True)

# these parameters are used to match up with the parameters for the VQE run on water in the paper: https://arxiv.org/pdf/2106.13840.pdf
active_electrons = 16
active_orbitals = 10

# define the hamiltonian needed to compute cost-function
def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons,
                                           active_orbitals = active_orbitals, method = "pyscf")[0]

hf = qml.qchem.hf_state(electrons=active_electrons, orbitals=active_orbitals*2)
print(hf)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]


In [ ]:
# Need to run this: functions to allow us to calculate angle between OH1 and OH2:
import math

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return (np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180/math.pi) #convert to degrees

def euclidean_to_z_matrix(x):
    x = x.numpy() # work with numpy arrays, rather then tensors
    
    O1H1 = x[3:6] - x[0:3]
    O1H2 = x[6:9] - x[0:3]
    H1O2 = x[9:12] - x[3:6]
    O2H3 = x[12:15] - x[9:12]
    O2H4 = x[15:18] - x[9:12]

    r1 = np.linalg.norm(O1H1)
    
    
    angle1 = angle_between(O1H1, O1H2)
    
    r2 = np.linalg.norm(H1O2)
    
    angle2 = 180 - angle_between(O1H1, H1O2)
    
    dihedr_angle2 = angle_between(np.cross(O1H1, O1H2), np.cross(O1H1, H1O2))
    
    angle3 = 180 - angle_between(O2H3, H1O2)
    
    dihedr_angle3 = 180 - angle_between(np.cross(O2H3, H1O2), np.cross(O1H1, H1O2))
    
    dihedr_angle4 = angle_between(np.cross(O2H4, O2H3), np.cross(O2H3, H1O2))
    
    return r1, angle1, r2, angle2, dihedr_angle2, angle3, dihedr_angle3, dihedr_angle4

In [ ]:
################ Code from the 'Building the adaptive circuit' tutorial below:################
active_electrons = active_electrons
active_orbitals = active_orbitals

H, qubits = qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons, active_orbitals = active_orbitals, method = "pyscf")
singles, doubles = qchem.excitations(active_electrons, qubits)

hf_state = qchem.hf_state(active_electrons, qubits)
print(hf_state)



#compute the significant double-excitation gates:
def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

# dev = qml.device("default.qubit", wires=qubits)
dev = qml.device("lightning.gpu", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev, interface="autograd", diff_method="adjoint")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)
print("Computed gradients for all possible Double Excitation Gates: \n")
for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")   
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected double-excitation gates: ", len(doubles_select))



# optimizing the parameters for the double-excitation gates for Ansatz-wavefunction construction
opt = qml.GradientDescentOptimizer(stepsize=0.5)
params_doubles = np.zeros(len(doubles_select), requires_grad=True)
for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)
print("Done!")



#compute the significant single-excitation gates:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)


cost_fn = qml.QNode(circuit_2, dev, interface="autograd", diff_method="adjoint")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)
grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)
print("Computed gradients for all possible Single Excitation Gates: \n")
for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected single-excitation gates: ", len(singles_select))

#Total Number of Gates selected to construct the Quantum Ansatz:
print("Total selected gates: "+  str(len(doubles_select) + len(singles_select)))

################# End code from the 'Building the adaptive circuit' tutorial ##################

In [ ]:
# End of ADAPT VQE ^ 
# Now use Soran's modified code down below:


symbols = ["O", "H", "H", "O", "H", "H"]
geometry = np.array([ 0.        ,  0.        ,  0.        ,  1.5633147 ,  0.98274071,
        -0.60341695, -0.7834994 ,  1.43656484,  1.04702164,  1.88208385,
         1.81779458, -0.08610321,  2.16704842,  2.60039367,  1.66894039,
         2.22750225,  3.42941152, -1.11431581], requires_grad=True)

mol = qml.qchem.Molecule(symbols, geometry)
# dev = qml.device("default.qubit")
dev = qml.device("lightning.qubit")

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

In [ ]:
def energy(mol):
    @qml.qnode(dev, interface="autograd")
    def circuit(*args):
        
        # note that active_electrons=2, active_orbitals=2 in this example
        qml.BasisState(hf_state, wires=range(qubits))
        
        # apply all single excitations
        for i, singles in enumerate(singles_select):
            qml.SingleExcitation(*args[0][i], wires=singles)
            
        # apply all double excitations
        for j, doubles in enumerate(doubles_select):
            qml.DoubleExcitation(*args[0][j + len(singles_select)], wires=doubles)
        
        # note that active_electrons=2, active_orbitals=2 in this example
        H = qml.qchem.molecular_hamiltonian(mol.symbols, mol.coordinates,
            active_electrons=active_electrons, active_orbitals=active_orbitals, args=args[1:], method = "pyscf")[0]
        
        return qml.expval(H)
    return circuit

In [ ]:
# number of zeros should match the number of circuit gates
circuit_param = [np.array([0.0], requires_grad=True)] * (len(singles_select) + len(doubles_select))
# THIS LINE MADE THE DIFFERENCE IN THE CODE:

# Calculate starting water-molecule parameters:
r1, angle1, r2, angle2, dihedr_angle2, angle3, dihedr_angle3, dihedr_angle4 = euclidean_to_z_matrix(x)
print("Starting z-matrix parameters:")
print(f"Bond r1 = {r1:.5f} A, Bond angle1 = {angle1:.5f}" + '\u00b0')
print(f"Bond r2 = {r2:.5f} A, Bond angle2 = {angle2:.5f}" + '\u00b0' + f", Bond Dihedr_angle2 = {dihedr_angle2:.5f}" + '\u00b0')
print(f"Bond angle3 = {angle3:.5f}" + '\u00b0' + f", Bond Dihedr_angle3 = {dihedr_angle3:.5f}" + '\u00b0')
print(f", Bond Dihedr_angle4 = {dihedr_angle4:.5f}" + '\u00b0')
print("")

energy_vec = []
bond_r1 = []
bond_angle1 = []
bond_r2 = []
bond_angle2 = []
bond_dihedr_angle2 = []
bond_angle3 = []
bond_dihedr_angle3 = []
bond_dihedr_angle4 = []

theta_learning_rate = 0.4
x_learning_rate = 0.5

eps = 1e-05
n = 0

start = time.time()

while True:
    args = [circuit_param, geometry]
    mol = qml.qchem.Molecule(symbols, geometry)

    g_param = grad(energy(mol), argnum = 0)(*args)
    circuit_param = circuit_param - theta_learning_rate * g_param[0]

    forces = -grad(energy(mol), argnum = 1)(*args)
    geometry = geometry + x_learning_rate * forces
    
    
    energy_recalc = energy(mol)(*args)
    print(f'n: {n}, E: {energy_recalc:.8f}, Force-max: {abs(forces).max():.8f}')
    
    # Calculate updated z-matrix parameters:
    r1, angle1, r2, angle2, dihedr_angle2, angle3, dihedr_angle3, angle4 = euclidean_to_z_matrix(x)
    
    # Add results to array:
    energy_vec.append(energy_recalc)
    bond_r1.append(r1)
    bond_angle1.append(angle1)
    bond_r2.append(r2)
    bond_angle2.append(angle2)
    bond_dihedr_angle2.append(dihedr_angle2)
    bond_angle3.append(angle3)
    bond_dihedr_angle3.append(dihedr_angle3)
    bond_dihedr_angle4.append(dihedr_angle4)
    
    print("Updated z-matrix parameters")
    print(f"Bond r1 = {r1:.5f} A, Bond angle1 = {angle1:.5f}" + '\u00b0')
    print(f"Bond r2 = {r2:.5f} A, Bond angle2 = {angle2:.5f}" + '\u00b0' + f", Bond Dihedr_angle2 = {dihedr_angle2:.5f}" + '\u00b0')
    print(f"Bond angle3 = {angle3:.5f}" + '\u00b0' + f", Bond Dihedr_angle3 = {dihedr_angle3:.5f}" + '\u00b0')
    print(f", Bond Dihedr_angle4 = {dihedr_angle4:.5f}" + '\u00b0')
    print("")
    
    n += 1
    if n <= 1:
        continue
    if np.max(np.abs(forces)) <= 1e-05 or np.abs(energy_vec[-2]-energy_vec[-1]) < eps:
        print("Successfully converged!")
        break
        
print("Total time:", time.time()-start, "seconds")

print("\n" f"Final value of the ground-state energy = {energy_vec[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[i][0]:.4f}   {x[i][1]:.4f}   {x[0][2]:.4f}")

In [ ]:
bond_length[-1], bond_angle[-1]

In [ ]:
circuit_param = [np.array([0.0], requires_grad=True)] * (len(singles_select) + len(doubles_select))

In [ ]:
def function(*args):
    print(*args[0][0])
function(*args)           

In [ ]:
import matplotlib.pyplot as plt
np_bl = np.array(bond_length)
np_ba = np.array(bond_angle)
np_en = np.array(energy_vec)
plt.plot(np_bl)
plt.xlabel("Steps, n")
plt.ylabel("Angstroms, Å")
plt.title("Bond length (r) optimization convergence")
plt.grid(True)

In [ ]:
plt.plot(np_ba)
plt.xlabel("Steps, n")
plt.ylabel("Degrees, \u00b0")
plt.title("Bond angle (\u00b0) optimization convergence water, e-= 8, mo orbitals=6")
plt.grid(True)

In [ ]:
plt.plot(np_en)
plt.xlabel("Steps, n")
plt.ylabel("Energy, Ha")
plt.title("Energy (Ha) optimization convergence water, e-= 8, mo orbitals=6")
plt.grid(True)

In [ ]:
plt.plot(np_bl, np_en)
plt.xlabel("Bond Length, n")
plt.ylabel("Energy, Ha")
plt.title("Bond Length vs Energy (Ha) optimization convergence water, e-= 8, mo orbitals=6")
plt.grid(True)

In [ ]:
plt.plot(np_ba, np_en)
plt.xlabel("Bond Angle, n")
plt.ylabel("Energy, Ha")
plt.title("Bond Angle vs Energy (Ha) optimization convergence water, e-= 8, mo orbitals=6")
plt.grid(True)